# Comparativo de novos casos por dia por estado [in Portuguese]
> Casos confirmados e mortes - novos casos por dia (linear scale)

- toc: false
- badges: true
- comments: true
- author: Cleber Jorge Amaral
- categories: [covid-19, brasil, comparativo, altair, jupyter]
- image: images/brazil-states-linear-cases-and-deaths-newcases.png

### NOTA
Os dados utilizados nestas análises são inerentemente incertos. Não há garantia de que os dados e as análises estejam devidamente atualizados e corretos. Antes de tomar conclusões é fundamental verificar dados e conclusões com outras fontes.

In [1]:
#hide
import pandas as pd
import altair as alt
import math
import requests
from altair_saver import save
from IPython.display import HTML

CHART_WIDTH = 600
CHART_HEIGHT = 400

In [2]:
#hide
STATE_COLUMN = "Estado"
DATE_COLUMN = "Data"
DEATH_COLUMN = "Mortes"
NEWCONFIRMED_CASES = "Novo Confirmado"
NEWDEATH_COLUMN = "Nova Morte"
NEWRECOVERY_COLUMN = "Novo Recuperado"
ACTIVE_COLUMN = "Ativo"
TOTAL_BY_STATE = "total por estado"
CONFIRMED_CASES = 'Casos confirmados'
DEATHS = 'Mortes'
NEW_CASES = 'novos casos por dia'
ACCUMULATED = 'acumulado'

In [3]:
#hide
url = ('https://data.brasil.io/dataset/covid19/caso_full.csv.gz')

# Pretending to be a browser
header = {
  "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/581.0.4044.138 Safari/537.36",
  "x-requested-with": "XMLHttpRequest",
  "accept-encoding": "gzip, deflate, br"
}

response = requests.get(url, headers=header, stream=True)

raw = response.raw
with open('caso_full.csv.gz', 'wb') as out_file:
    while True:
        chunk = raw.read(1024, decode_content=True)
        if not chunk:
            break
        out_file.write(chunk)

src = pd.read_csv('caso_full.csv.gz')

df = src[(src['place_type'] == 'state')]
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.strftime('%m/%d/%y')

df = df.rename(columns={'date': DATE_COLUMN, 'state': STATE_COLUMN, 'new_confirmed': NEWCONFIRMED_CASES, 'new_deaths': NEWDEATH_COLUMN})
#df.head()

/home/cleber/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/cleber/.local/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [4]:
#hide
LAST_DATE = max(df[DATE_COLUMN])

In [5]:
#hide_input
opt = df[STATE_COLUMN].unique()
opt.sort()
input_dropdown = alt.binding_select(options=opt)
selection1 = alt.selection_single(fields=[STATE_COLUMN], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=[STATE_COLUMN], on='mouseover')
color = alt.condition(selection1 | selection2,
                    alt.Color(STATE_COLUMN+':N', scale=alt.Scale(scheme='tableau20'), legend=None),
                    alt.value('lightgray'))

chart = alt.Chart(df).mark_bar().encode(
    x=alt.X(DATE_COLUMN+':O', axis=alt.Axis(title=DATE_COLUMN)),
    y=alt.Y(NEWCONFIRMED_CASES, axis=alt.Axis(title=NEWCONFIRMED_CASES)),
    color=color,
    tooltip=[DATE_COLUMN, STATE_COLUMN, NEWCONFIRMED_CASES],
    order=alt.Order(
    STATE_COLUMN,
    sort='descending'
    )
).properties(
    title=CONFIRMED_CASES+": "+TOTAL_BY_STATE+' ('+NEW_CASES+')'
).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

legend = alt.Chart(df).mark_point().encode(
    y=alt.Y(STATE_COLUMN+':N', axis=alt.Axis(orient='right')),
    color=color
).add_selection(
    selection1, selection2
)

plot = chart.properties(width=CHART_WIDTH, height=CHART_HEIGHT) | legend
plot

alt.HConcatChart(...)

In [6]:
#hide_input
opt = df[STATE_COLUMN].unique()
opt.sort()
input_dropdown = alt.binding_select(options=opt)
selection1 = alt.selection_single(fields=[STATE_COLUMN], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=[STATE_COLUMN], on='mouseover')
color = alt.condition(selection1 | selection2,
                    alt.Color(STATE_COLUMN+':N', scale=alt.Scale(scheme='tableau20'), legend=None),
                    alt.value('lightgray'))

chart = alt.Chart(df).mark_bar().encode(
    x=alt.X(DATE_COLUMN+':O', axis=alt.Axis(title=DATE_COLUMN)),
    y=alt.Y(NEWDEATH_COLUMN, axis=alt.Axis(title=NEWDEATH_COLUMN)),
    color=color,
    tooltip=[DATE_COLUMN, STATE_COLUMN, NEWDEATH_COLUMN],
    order=alt.Order(
    STATE_COLUMN,
    sort='descending'
    )
).properties(
    title=DEATHS+": "+TOTAL_BY_STATE+' ('+NEW_CASES+')'
).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

legend = alt.Chart(df).mark_point().encode(
    y=alt.Y(STATE_COLUMN+':N', axis=alt.Axis(orient='right')),
    color=color
).add_selection(
    selection1, selection2
)

chart.properties(width=CHART_WIDTH, height=CHART_HEIGHT) | legend

alt.HConcatChart(...)

In [7]:
#hide_input
HTML(f'<small class="float-right">Última atualização em {pd.to_datetime(LAST_DATE).strftime("%d/%m/%Y")}</small>')

Based on works developed by [Alonso Silva Allende](https://covid19dashboards.com/jupyter/2020/04/27/Covid-19-Overview-Chile.html), adapted by [Cleber Jorge Amaral](http://cleberjamaral.github.io/). Data source: [brasil.io](https://brasil.io/home/)

In [8]:
#hide
save(plot,"../images/brazil-states-linear-cases-and-deaths-newcases.png")

ValueError: No enabled saver found that supports format='png'